# Code for running the regression for the impact of the flow of momentum on managerial decision-making

In [ ]:
#import the regression dataset
import pickle
infile = open('input PC regression Momentum on decisions.p','rb')
df_mom = pickle.load(infile)
infile.close()

In [ ]:
#calcualte the score differences
df_mom['score difference home'] = df_mom['Home score'] - df_mom['Away score']
df_mom['score difference away'] = df_mom['Away score'] - df_mom['Home score']

In [ ]:
#add the week in which the matches are played
import pandas as pd
from progressbar import ProgressBar
match_weeks = []
df_weeks = pd.read_excel('dfweeks.xlsx')
pbar = ProgressBar()
for i in pbar(range(len(df_mom))):
    for j in range(len(df_weeks)):
        if df_mom.iloc[i]['Match'] == str(df_weeks.iloc[j]['match']):
            match_weeks.append(df_weeks.iloc[j]['week'])
df_mom['week'] = match_weeks

In [ ]:
#add the type of decision (response variable)
type_decision = []
for i in range(len(df_mom)):
    if df_mom.iloc[i]['type decision'] == 'offensive':
        type_decision.append(1)
    elif df_mom.iloc[i]['type decision'] == 'defensive':
        type_decision.append(-1)
    else:
        type_decision.append(0)
df_mom['type decision'] = type_decision
    

In [ ]:
#select necessary columns in the dataframe
import pandas as pd
df_mom = df_mom.dropna()
df = pd.DataFrame(df_mom,columns=['delta mom pc home','delta mom pc away', 'type decision', 'average mom pc home', 'average mom pc away', 'average mom all pc home', 'average mom all pc away', 'score difference home', 'score difference away', 'Time left', 'week', 'last goal',  'var mom pc home', 'var mom pc away', 'team'])

In [ ]:
#regression model PC
import numpy as np
import pandas as pd
import scipy.stats as stats
import pickle
from progressbar import ProgressBar
from bevel.linear_ordinal_regression import OrderedLogit
params = []
pbar = ProgressBar()
#iterate the regression a thousand times
for i in pbar(range(1000)):
    deltamom = []
    type_decision = []
    mean_mom_before = []
    mean_mom_all = []
    score_difference = []
    time_left = []
    week = []
    previous_goal = []
    varmom = []
    #resample the type of decision
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    #if the decision is made by the home team: add home variables. Else, add away varaibles
    for j in range(len(df_new)):
        if df_new.iloc[j]['team'] == 'Home':
            deltamom.append(df_new.iloc[j]['delta mom pc home'])
            mean_mom_before.append(df_new.iloc[j]['average mom pc home'])
            mean_mom_all.append(df_new.iloc[j]['average mom all pc home'])
            endpc.append(df_new.iloc[j]['end pc home'])
            score_difference.append(df_new.iloc[j]['score difference home'])
            deltapc.append(df_new.iloc[j]['delta pc home'])
            varmom.append(df_new.iloc[j]['var mom pc home'])
            varpc.append(df_new.iloc[j]['var pc home'])
        else:
            deltamom.append(df_new.iloc[j]['delta mom pc away'])
            mean_mom_before.append(df_new.iloc[j]['average mom pc away'])
            mean_mom_all.append(df_new.iloc[j]['average mom all pc away'])
            endpc.append(df_new.iloc[j]['end pc away'])
            score_difference.append(df_new.iloc[j]['score difference away'])
            deltapc.append(df_new.iloc[j]['delta pc away'])
            varmom.append(df_new.iloc[j]['var mom pc away'])
            varpc.append(df_new.iloc[j]['var pc away'])
        
        type_decision.append(df_new.iloc[j]['type decision'])
        time_left.append(df_new.iloc[j]['Time left'])
        week.append(df_new.iloc[j]['week'])
        previous_goal.append(df_new.iloc[j]['last goal'])
            

    #create dataframe for regession
    data = {'delta mom': deltamom,
           'meanmombefore': mean_mom_before,
           'meanmomall': mean_mom_all,
           'score difference': score_difference,
           'time left': time_left,
           'week': week,
           'previous goal': previous_goal,
           'varmom': varmom,
           'type decision': type_decision}
    df_reg = pd.DataFrame(data)
    
    X = df_reg[['delta mom', 'meanmombefore', 'meanmomall', 'score difference', 'time left', 'week', 'previous goal', 'varmom']]
    y = df_reg['type decision']

    #Run ordinal regression
    ol = OrderedLogit()
    model = ol.fit(X, y)

    #add beta params to the list
    params.append(model.beta_)
data = {'beta params': params}

df_pc = pd.DataFrame(data)
     

In [ ]:
#For the empirical distributions, calculate the mode of the beta coefficients and fnd the confidence intervals.
#The other variables are calculated from these distributions and coefficients
#the p-values are found in the table for the t-test
import statistics
deltamom_pc = []
meanmombefore_pc = []
meanmomall_pc = []
score_difference_pc = []
time_left_pc = []
week_pc = []
previous_goal_pc = []
varmompc = []
for i in range(len(df_pc)):
    deltamom_pc.append(df_pc.iloc[i]['beta params'][0])
    meanmombefore_pc.append(df_pc.iloc[i]['beta params'][1])
    meanmomall_pc.append(df_pc.iloc[i]['beta params'][2])
    score_difference_pc.append(df_pc.iloc[i]['beta params'][3])
    time_left_pc.append(df_pc.iloc[i]['beta params'][4])
    week_pc.append(df_pc.iloc[i]['beta params'][5])
    previous_goal_pc.append(df_pc.iloc[i]['beta params'][6])
    varmompc.append(df_pc.iloc[i]['beta params'][7])

deltamom_pc = [round(i,4) for i in deltamom_pc]
meanmombefore_pc = [round(i,4) for i in meanmombefore_pc]
meanmomall_pc = [round(i,4) for i in meanmomall_pc]
score_difference_pc = [round(i,4) for i in score_difference_pc]
time_left_pc = [round(i,4) for i in time_left_pc]
week_pc = [round(i,4) for i in week_pc]
previous_goal_pc = [round(i,4) for i in previous_goal_pc]
varmompc = [round(i,4) for i in varmompc]

#betavariabelen
from statistics import mean
beta_deltamom_pc = mean(statistics.multimode(deltamom_pc))
beta_meanmombefore_pc = mean(statistics.multimode(meanmombefore_pc))
beta_meanmomall_pc = mean(statistics.multimode(meanmomall_pc))
beta_score_difference_pc = mean(statistics.multimode(score_difference_pc))
beta_time_left_pc = mean(statistics.multimode(time_left_pc))
beta_week_pc = mean(statistics.multimode(week_pc))
beta_previous_goal_pc = mean(statistics.multimode(previous_goal_pc))
beta_varmompc = mean(statistics.multimode(varmompc))

confidence_intervals_deltamom_pc = [sorted(deltamom_pc)[25], sorted(deltamom_pc)[974]]
confidence_intervals_meanmombefore_pc = [sorted(meanmombefore_pc)[25], sorted(meanmombefore_pc)[974]]
confidence_intervals_meanmomall_pc = [sorted(meanmomall_pc)[25], sorted(meanmomall_pc)[974]]
confidence_intervals_score_difference_pc = [sorted(score_difference_pc)[25], sorted(score_difference_pc)[974]]
confidence_intervals_time_left_pc = [sorted(time_left_pc)[25], sorted(time_left_pc)[974]]
confidence_intervals_week_pc = [sorted(week_pc)[25], sorted(week_pc)[974]]
confidence_intervals_previous_goal_pc = [sorted(previous_goal_pc)[25], sorted(previous_goal_pc)[974]]
confidence_intervals_varmompc = [sorted(varmompc)[25], sorted(varmompc)[974]]

SE_deltamom_pc = (confidence_intervals_deltamom_pc[1] - confidence_intervals_deltamom_pc[0])/(2*1.96)
SE_meanmombefore_pc = (confidence_intervals_meanmombefore_pc[1] - confidence_intervals_meanmombefore_pc[0])/(2*1.96)
SE_meanmomall_pc = (confidence_intervals_meanmomall_pc[1] - confidence_intervals_meanmomall_pc[0])/(2*1.96)
SE_score_difference_pc = (confidence_intervals_score_difference_pc[1] - confidence_intervals_score_difference_pc[0])/(2*1.96)
SE_time_left_pc = (confidence_intervals_time_left_pc[1] - confidence_intervals_time_left_pc[0])/(2*1.96)
SE_week_pc = (confidence_intervals_week_pc[1] - confidence_intervals_week_pc[0])/(2*1.96)
SE_previous_goal_pc = (confidence_intervals_previous_goal_pc[1] - confidence_intervals_previous_goal_pc[0])/(2*1.96)
SE_varmompc = (confidence_intervals_varmompc[1] - confidence_intervals_varmompc[0])/(2*1.96)

t_deltamom_pc = beta_deltamom_pc/SE_deltamom_pc
t_meanmombefore_pc = beta_meanmombefore_pc/SE_meanmombefore_pc
t_meanmomall_pc = beta_meanmomall_pc/SE_meanmomall_pc
t_score_difference_pc = beta_score_difference_pc/SE_score_difference_pc
t_time_left_pc = beta_time_left_pc/SE_time_left_pc
t_week_pc = beta_week_pc/SE_week_pc
t_previous_goal_pc = beta_previous_goal_pc/SE_previous_goal_pc
t_varmompc = beta_varmompc/SE_varmompc

degrees_freedom_residuals_pc = len(df_test) - 10 - 1
    

In [ ]:
#calculations for the PCV model
#these are the same as in the PC model
import pandas as pd
df_mom = df_mom.dropna()
df = pd.DataFrame(df_mom,columns=['delta mom pcv home','delta mom pcv away', 'type decision', 'average mom pcv home', 'average mom pcv away', 'average mom all pcv home', 'average mom all pcv away', 'score difference home', 'score difference away', 'Time left', 'week', 'last goal', 'var mom pcv home', 'var mom pcv away', 'team'])

In [ ]:
#pcv regression
import numpy as np
import pandas as pd
import scipy.stats as stats
import pickle
from progressbar import ProgressBar
from bevel.linear_ordinal_regression import OrderedLogit
params = []
pbar = ProgressBar()
for i in pbar(range(1000)):
    deltamom = []
    type_decision = []
    mean_mom_before = []
    mean_mom_all = []
    score_difference = []
    time_left = []
    week = []
    previous_goal = []]
    varmom = []
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        if df_new.iloc[j]['team'] == 'Home':
            deltamom.append(df_new.iloc[j]['delta mom pcv home'])
            mean_mom_before.append(df_new.iloc[j]['average mom pcv home'])
            mean_mom_all.append(df_new.iloc[j]['average mom all pcv home'])
            score_difference.append(df_new.iloc[j]['score difference home'])
            deltapcv.append(df_new.iloc[j]['delta pcv home'])
            varmom.append(df_new.iloc[j]['var mom pcv home'])
        else:
            deltamom.append(df_new.iloc[j]['delta mom pcv away'])
            mean_mom_before.append(df_new.iloc[j]['average mom pcv away'])
            mean_mom_all.append(df_new.iloc[j]['average mom all pcv away'])
            score_difference.append(df_new.iloc[j]['score difference away'])
            varmom.append(df_new.iloc[j]['var mom pcv away'])
        
        type_decision.append(df_new.iloc[j]['type decision'])
        time_left.append(df_new.iloc[j]['Time left'])
        week.append(df_new.iloc[j]['week'])
        previous_goal.append(df_new.iloc[j]['last goal'])
            

    data = {'delta mom': deltamom,
           'meanmombefore': mean_mom_before,
           'meanmomall': mean_mom_all,
           'score difference': score_difference,
           'time left': time_left,
           'week': week,
           'previous goal': previous_goal,
           'varmom': varmom,
           'type decision': type_decision}
    df_reg = pd.DataFrame(data)
    
    X = df_reg[['delta mom', 'meanmombefore', 'meanmomall', 'score difference', 'time left', 'week', 'previous goal', 'varmom']]
    y = df_reg['type decision']

    # Note the difference in argument order
    ol = OrderedLogit()
    model = ol.fit(X, y)

    # Print out the statistics
    params.append(model.beta_)
data = {'beta params': params}

df_pcv = pd.DataFrame(data)
     

In [ ]:
#calculate statistics for the pcv regression
import statistics
deltamom_pcv = []
meanmombefore_pcv = []
meanmomall_pcv = []
score_difference_pcv = []
time_left_pcv = []
week_pcv = []
previous_goal_pcv = []
varmompcv = []
for i in range(len(df_pcv)):
    deltamom_pcv.append(df_pcv.iloc[i]['beta params'][0])
    meanmombefore_pcv.append(df_pcv.iloc[i]['beta params'][1])
    meanmomall_pcv.append(df_pcv.iloc[i]['beta params'][2])
    score_difference_pcv.append(df_pcv.iloc[i]['beta params'][3])
    time_left_pcv.append(df_pcv.iloc[i]['beta params'][4])
    week_pcv.append(df_pcv.iloc[i]['beta params'][5])
    previous_goal_pcv.append(df_pcv.iloc[i]['beta params'][6])
    varmompcv.append(df_pcv.iloc[i]['beta params'][7])

deltamom_pcv = [round(i,4) for i in deltamom_pcv]
meanmombefore_pcv = [round(i,4) for i in meanmombefore_pcv]
meanmomall_pcv = [round(i,4) for i in meanmomall_pcv]
score_difference_pcv = [round(i,4) for i in score_difference_pcv]
time_left_pcv = [round(i,4) for i in time_left_pcv]
week_pcv = [round(i,4) for i in week_pcv]
previous_goal_pcv = [round(i,4) for i in previous_goal_pcv]
varmompcv = [round(i,4) for i in varmompcv]

#betavariabelen
from statistics import mean
beta_deltamom_pcv = mean(statistics.multimode(deltamom_pcv))
beta_meanmombefore_pcv = mean(statistics.multimode(meanmombefore_pcv))
beta_meanmomall_pcv = mean(statistics.multimode(meanmomall_pcv))
beta_score_difference_pcv = mean(statistics.multimode(score_difference_pcv))
beta_time_left_pcv = mean(statistics.multimode(time_left_pcv))
beta_week_pcv = mean(statistics.multimode(week_pcv))
beta_previous_goal_pcv = mean(statistics.multimode(previous_goal_pcv))
beta_varmompcv = mean(statistics.multimode(varmompcv))

confidence_intervals_deltamom_pcv = [sorted(deltamom_pcv)[25], sorted(deltamom_pcv)[974]]
confidence_intervals_meanmombefore_pcv = [sorted(meanmombefore_pcv)[25], sorted(meanmombefore_pcv)[974]]
confidence_intervals_meanmomall_pcv = [sorted(meanmomall_pcv)[25], sorted(meanmomall_pcv)[974]]
confidence_intervals_score_difference_pcv = [sorted(score_difference_pcv)[25], sorted(score_difference_pcv)[974]]
confidence_intervals_time_left_pcv = [sorted(time_left_pcv)[25], sorted(time_left_pcv)[974]]
confidence_intervals_week_pcv = [sorted(week_pcv)[25], sorted(week_pcv)[974]]
confidence_intervals_previous_goal_pcv = [sorted(previous_goal_pcv)[25], sorted(previous_goal_pcv)[974]]
confidence_intervals_varmompcv = [sorted(varmompcv)[25], sorted(varmompcv)[974]]

SE_deltamom_pcv = (confidence_intervals_deltamom_pcv[1] - confidence_intervals_deltamom_pcv[0])/(2*1.96)
SE_meanmombefore_pcv = (confidence_intervals_meanmombefore_pcv[1] - confidence_intervals_meanmombefore_pcv[0])/(2*1.96)
SE_meanmomall_pcv = (confidence_intervals_meanmomall_pcv[1] - confidence_intervals_meanmomall_pcv[0])/(2*1.96)
SE_score_difference_pcv = (confidence_intervals_score_difference_pcv[1] - confidence_intervals_score_difference_pcv[0])/(2*1.96)
SE_time_left_pcv = (confidence_intervals_time_left_pcv[1] - confidence_intervals_time_left_pcv[0])/(2*1.96)
SE_week_pcv = (confidence_intervals_week_pcv[1] - confidence_intervals_week_pcv[0])/(2*1.96)
SE_previous_goal_pcv = (confidence_intervals_previous_goal_pcv[1] - confidence_intervals_previous_goal_pcv[0])/(2*1.96)
SE_varmompcv = (confidence_intervals_varmompcv[1] - confidence_intervals_varmompcv[0])/(2*1.96)

t_deltamom_pcv = beta_deltamom_pcv/SE_deltamom_pcv
t_meanmombefore_pcv = beta_meanmombefore_pcv/SE_meanmombefore_pcv
t_meanmomall_pcv = beta_meanmomall_pcv/SE_meanmomall_pcv
t_score_difference_pcv = beta_score_difference_pcv/SE_score_difference_pcv
t_time_left_pcv = beta_time_left_pcv/SE_time_left_pcv
t_week_pcv = beta_week_pcv/SE_week_pcv
t_previous_goal_pcv = beta_previous_goal_pcv/SE_previous_goal_pcv
t_varmompcv = beta_varmompcv/SE_varmompcv

degrees_freedom_residuals_pcv = len(df_test) - 10 - 1
    

In [ ]:
#regression preparations xT
#these are the same as for pc and pcv
import pickle
infile = open('input xT regression Momentum on decisions.p','rb')
df_mom = pickle.load(infile)
infile.close()

df_mom['score difference home'] = df_mom['Home score'] - df_mom['Away score']
df_mom['score difference away'] = df_mom['Away score'] - df_mom['Home score']

import pandas as pd
from progressbar import ProgressBar
match_weeks = []
df_weeks = pd.read_excel('dfweeks.xlsx')
pbar = ProgressBar()
for i in pbar(range(len(df_mom))):
    for j in range(len(df_weeks)):
        if df_mom.iloc[i]['Match'] == str(df_weeks.iloc[j]['match']):
            match_weeks.append(df_weeks.iloc[j]['week'])
df_mom['week'] = match_weeks

infile = open('shots and goals event dataframes.p','rb')
df_shots = pickle.load(infile)
infile.close()

matches = df_mom.Match.unique()
last_goal = []
for i in range(len(matches)):
    df_match_mom = df_mom.loc[df_mom['Match'] == matches[i]]
    df_match_shots = df_shots.loc[df_shots['match_id'] == matches[i]]
    goals = []
    for j in range(len(df_match_shots)):
        if df_match_shots.iloc[j]['type_id'] == '16':
            goals.append(int(df_match_shots.iloc[j]['minute']))

    for j in range(len(df_match_mom)):
        goals_now = [i for i in goals if i<int(df_match_mom.iloc[j]['minute'])]
        if len(goals_now) == 0:
            last_goal.append(int(df_match_mom.iloc[j]['minute']))
        else:
            last_goal.append(int(df_match_mom.iloc[j]['minute']) - goals_now[-1])

df_mom['last goal'] = last_goal

type_decision = []
for i in range(len(df_mom)):
    if df_mom.iloc[i]['type decision'] == 'offensive':
        type_decision.append(1)
    elif df_mom.iloc[i]['type decision'] == 'defensive':
        type_decision.append(-1)
    else:
        type_decision.append(0)
df_mom['type decision'] = type_decision
    

In [ ]:
#select the necessary columns
import pandas as pd
df_mom = df_mom.dropna()
df = pd.DataFrame(df_mom,columns=['delta momentum home previous 10 min','delta momentum away previous 10 min', 'type decision', 'mean momentum home previous 10 min', 'mean momentum away previous 10 min', 'mean momentum home all minutes', 'mean momentum away all minutes', 'score difference home', 'score difference away', 'Time left', 'week', 'last goal', 'var momentum home previous 10 min', 'var momentum away previous 10 min', 'team'])

In [ ]:
#run the xT regresison
import numpy as np
import pandas as pd
import scipy.stats as stats
import pickle
from progressbar import ProgressBar
from bevel.linear_ordinal_regression import OrderedLogit
params = []
pbar = ProgressBar()
for i in pbar(range(1000)):
    deltamom = []
    type_decision = []
    mean_mom_before = []
    mean_mom_all = []
    score_difference = []
    time_left = []
    week = []
    previous_goal = []
    varmom = []
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        if df_new.iloc[j]['team'] == 'Home':
            deltamom.append(df_new.iloc[j]['delta momentum home previous 10 min'])
            mean_mom_before.append(df_new.iloc[j]['mean momentum home previous 10 min'])
            mean_mom_all.append(df_new.iloc[j]['mean momentum home all minutes'])
            score_difference.append(df_new.iloc[j]['score difference home'])
            varmom.append(df_new.iloc[j]['var momentum home previous 10 min'])
        else:
            deltamom.append(df_new.iloc[j]['delta momentum away previous 10 min'])
            mean_mom_before.append(df_new.iloc[j]['mean momentum away previous 10 min'])
            mean_mom_all.append(df_new.iloc[j]['mean momentum away all minutes'])
            score_difference.append(df_new.iloc[j]['score difference away'])
            varmom.append(df_new.iloc[j]['var momentum away previous 10 min'])
        
        type_decision.append(df_new.iloc[j]['type decision'])
        time_left.append(df_new.iloc[j]['Time left'])
        week.append(df_new.iloc[j]['week'])
        previous_goal.append(df_new.iloc[j]['last goal'])
            

    data = {'delta mom': deltamom,
           'meanmombefore': mean_mom_before,
           'meanmomall': mean_mom_all,
           'score difference': score_difference,
           'time left': time_left,
           'week': week,
           'previous goal': previous_goal,
           'varmom': varmom,
           'type decision': type_decision}
    df_reg = pd.DataFrame(data)
    
    X = df_reg[['delta mom', 'meanmombefore', 'meanmomall', 'score difference', 'time left', 'week', 'previous goal', 'varmom']]
    y = df_reg['type decision']

    # Note the difference in argument order
    ol = OrderedLogit()
    model = ol.fit(X, y)

    # Print out the statistics
    params.append(model.beta_)
data = {'beta params': params}

df_xt = pd.DataFrame(data)
     

In [ ]:
#calculations of the statistics
import statistics
deltamom_xt = []
meanmombefore_xt = []
meanmomall_xt = []
score_difference_xt = []
time_left_xt = []
week_xt = []
previous_goal_xt = []
varmomxt = []
for i in range(len(df_xt)):
    deltamom_xt.append(df_xt.iloc[i]['beta params'][0])
    meanmombefore_xt.append(df_xt.iloc[i]['beta params'][1])
    meanmomall_xt.append(df_xt.iloc[i]['beta params'][2])
    score_difference_xt.append(df_xt.iloc[i]['beta params'][3])
    time_left_xt.append(df_xt.iloc[i]['beta params'][4])
    week_xt.append(df_xt.iloc[i]['beta params'][5])
    previous_goal_xt.append(df_xt.iloc[i]['beta params'][6])
    varmomxt.append(df_xt.iloc[i]['beta params'][7])

deltamom_xt = [round(i,4) for i in deltamom_xt]
meanmombefore_xt = [round(i,4) for i in meanmombefore_xt]
meanmomall_xt = [round(i,4) for i in meanmomall_xt]
score_difference_xt = [round(i,4) for i in score_difference_xt]
time_left_xt = [round(i,4) for i in time_left_xt]
week_xt = [round(i,4) for i in week_xt]
previous_goal_xt = [round(i,4) for i in previous_goal_xt]
varmomxt = [round(i,4) for i in varmomxt]

#betavariabelen
from statistics import mean
beta_deltamom_xt = mean(statistics.multimode(deltamom_xt))
beta_meanmombefore_xt = mean(statistics.multimode(meanmombefore_xt))
beta_meanmomall_xt = mean(statistics.multimode(meanmomall_xt))
beta_score_difference_xt = mean(statistics.multimode(score_difference_xt))
beta_time_left_xt = mean(statistics.multimode(time_left_xt))
beta_week_xt = mean(statistics.multimode(week_xt))
beta_previous_goal_xt = mean(statistics.multimode(previous_goal_xt))
beta_varmomxt = mean(statistics.multimode(varmomxt))

confidence_intervals_deltamom_xt = [sorted(deltamom_xt)[25], sorted(deltamom_xt)[974]]
confidence_intervals_meanmombefore_xt = [sorted(meanmombefore_xt)[25], sorted(meanmombefore_xt)[974]]
confidence_intervals_meanmomall_xt = [sorted(meanmomall_xt)[25], sorted(meanmomall_xt)[974]]
confidence_intervals_score_difference_xt = [sorted(score_difference_xt)[25], sorted(score_difference_xt)[974]]
confidence_intervals_time_left_xt = [sorted(time_left_xt)[25], sorted(time_left_xt)[974]]
confidence_intervals_week_xt = [sorted(week_xt)[25], sorted(week_xt)[974]]
confidence_intervals_previous_goal_xt = [sorted(previous_goal_xt)[25], sorted(previous_goal_xt)[974]]
confidence_intervals_varmomxt = [sorted(varmomxt)[25], sorted(varmomxt)[974]]

SE_deltamom_xt = (confidence_intervals_deltamom_xt[1] - confidence_intervals_deltamom_xt[0])/(2*1.96)
SE_meanmombefore_xt = (confidence_intervals_meanmombefore_xt[1] - confidence_intervals_meanmombefore_xt[0])/(2*1.96)
SE_meanmomall_xt = (confidence_intervals_meanmomall_xt[1] - confidence_intervals_meanmomall_xt[0])/(2*1.96)
SE_score_difference_xt = (confidence_intervals_score_difference_xt[1] - confidence_intervals_score_difference_xt[0])/(2*1.96)
SE_time_left_xt = (confidence_intervals_time_left_xt[1] - confidence_intervals_time_left_xt[0])/(2*1.96)
SE_week_xt = (confidence_intervals_week_xt[1] - confidence_intervals_week_xt[0])/(2*1.96)
SE_previous_goal_xt = (confidence_intervals_previous_goal_xt[1] - confidence_intervals_previous_goal_xt[0])/(2*1.96)
SE_varmomxt = (confidence_intervals_varmomxt[1] - confidence_intervals_varmomxt[0])/(2*1.96)

t_deltamom_xt = beta_deltamom_xt/SE_deltamom_xt
t_meanmombefore_xt = beta_meanmombefore_xt/SE_meanmombefore_xt
t_meanmomall_xt = beta_meanmomall_xt/SE_meanmomall_xt
t_score_difference_xt = beta_score_difference_xt/SE_score_difference_xt
t_time_left_xt = beta_time_left_xt/SE_time_left_xt
t_week_xt = beta_week_xt/SE_week_xt
t_previous_goal_xt = beta_previous_goal_xt/SE_previous_goal_xt
t_varmomxt = beta_varmomxt/SE_varmomxt

degrees_freedom_residuals_xt = len(df_test) - 10 - 1
    